<a href="https://colab.research.google.com/github/Masciel-Sevilla/Segmentacion/blob/main/test_fps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np
import time
import os
from glob import glob
from tensorflow.keras import layers

# =============================================================================
# DEFINIR LAS CLASES PERSONALIZADAS (necesarias para ambos modelos)
# =============================================================================

class ASPPModule(layers.Layer):
    def __init__(self, filters=192, **kwargs):
        super(ASPPModule, self).__init__(**kwargs)
        self.filters = filters
        self.conv_1x1 = layers.Conv2D(filters, 1, padding='same', use_bias=False)
        self.bn_1x1 = layers.BatchNormalization()
        self.relu_1x1 = layers.ReLU()
        self.conv_3x3_6 = layers.Conv2D(filters, 3, padding='same', dilation_rate=6, use_bias=False)
        self.bn_3x3_6 = layers.BatchNormalization()
        self.relu_3x3_6 = layers.ReLU()
        self.conv_3x3_12 = layers.Conv2D(filters, 3, padding='same', dilation_rate=12, use_bias=False)
        self.bn_3x3_12 = layers.BatchNormalization()
        self.relu_3x3_12 = layers.ReLU()
        self.conv_3x3_18 = layers.Conv2D(filters, 3, padding='same', dilation_rate=18, use_bias=False)
        self.bn_3x3_18 = layers.BatchNormalization()
        self.relu_3x3_18 = layers.ReLU()
        self.global_avg_pool = layers.GlobalAveragePooling2D(keepdims=True)
        self.conv_1x1_gap = layers.Conv2D(filters, 1, padding='same', use_bias=False)
        self.bn_1x1_gap = layers.BatchNormalization()
        self.relu_1x1_gap = layers.ReLU()
        self.conv_final = layers.Conv2D(filters, 1, padding='same', use_bias=False)
        self.bn_final = layers.BatchNormalization()
        self.relu_final = layers.ReLU()
        self.dropout = layers.Dropout(0.2)

    def call(self, inputs, training=None):
        input_shape = tf.shape(inputs)
        conv_1x1 = self.relu_1x1(self.bn_1x1(self.conv_1x1(inputs), training=training))
        conv_3x3_6 = self.relu_3x3_6(self.bn_3x3_6(self.conv_3x3_6(inputs), training=training))
        conv_3x3_12 = self.relu_3x3_12(self.bn_3x3_12(self.conv_3x3_12(inputs), training=training))
        conv_3x3_18 = self.relu_3x3_18(self.bn_3x3_18(self.conv_3x3_18(inputs), training=training))
        gap = self.global_avg_pool(inputs)
        gap = self.relu_1x1_gap(self.bn_1x1_gap(self.conv_1x1_gap(gap), training=training))
        gap = tf.image.resize(gap, [input_shape[1], input_shape[2]], method='bilinear')
        concat = layers.Concatenate()([conv_1x1, conv_3x3_6, conv_3x3_12, conv_3x3_18, gap])
        output = self.relu_final(self.bn_final(self.conv_final(concat), training=training))
        output = self.dropout(output, training=training)
        return output

    def get_config(self):
        config = super(ASPPModule, self).get_config()
        config.update({"filters": self.filters})
        return config

class DeformableAttention(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(DeformableAttention, self).__init__(**kwargs)
        self.filters = filters
        self.attention_conv = layers.Conv2D(self.filters, 1, padding='same', activation='sigmoid', name='attention_weights_conv', use_bias=False)
        self.bn_attention = layers.BatchNormalization()
        self.feature_conv = layers.SeparableConv2D(self.filters, 3, padding='same', name='feature_processing_conv', use_bias=False)
        self.bn_feature = layers.BatchNormalization()
        self.relu_feature = layers.ReLU()

    def call(self, inputs, training=None):
        attention_weights = self.bn_attention(self.attention_conv(inputs), training=training)
        features = self.relu_feature(self.bn_feature(self.feature_conv(inputs), training=training))
        attended_features = features * attention_weights
        return attended_features

    def get_config(self):
        config = super(DeformableAttention, self).get_config()
        config.update({"filters": self.filters})
        return config

# =============================================================================
# CONFIGURACIÓN
# =============================================================================
# --- ¡NUEVO! Lista de modelos a probar ---
MODELS_TO_TEST = [
    {'name': 'Modelo S (EfficientNetV2-S)', 'path': 'efficient_weed_model_S_best.keras'},
    {'name': 'Modelo B0 (EfficientNetV2-B0)', 'path': 'efficient_weed_model_B0_best.keras'}
]

TEST_IMAGES_PATH = './Balanced/test/images'
IMG_HEIGHT, IMG_WIDTH = 128, 128
NUM_IMAGES_TO_TEST = 100

# =============================================================================
# SCRIPT PRINCIPAL
# =============================================================================
print("--- Iniciando Test de Inferencia de FPS en Google Colab ---")

# Verificar GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detectada: {gpus[0].name.split(':')[-1]}")
else:
    print("❌ No se detectó ninguna GPU.")

# Cargar imágenes (una sola vez para todos los modelos)
print(f"\nCargando {NUM_IMAGES_TO_TEST} imágenes de prueba...")
image_paths = sorted(glob(os.path.join(TEST_IMAGES_PATH, '*.jpg')))[:NUM_IMAGES_TO_TEST]
test_images = [tf.keras.applications.efficientnet_v2.preprocess_input(tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(path), channels=3), [IMG_HEIGHT, IMG_WIDTH])) for path in image_paths]
test_images_batch = np.array(test_images)
print(f"Imágenes cargadas. Shape del batch: {test_images_batch.shape}")

# --- ¡NUEVO! Bucle para probar cada modelo ---
for model_info in MODELS_TO_TEST:
    model_name = model_info['name']
    model_path = model_info['path']

    print("\n" + "="*50)
    print(f"Analizando: {model_name}")
    print("="*50)

    if not os.path.exists(model_path):
        print(f"❌ ERROR: No se encontró el archivo del modelo: {model_path}. Omitiendo.")
        continue

    # Cargar el modelo actual
    print(f"Cargando modelo: {model_path}...")
    custom_objects = {'ASPPModule': ASPPModule, 'DeformableAttention': DeformableAttention}
    model = tf.keras.models.load_model(model_path, custom_objects=custom_objects, compile=False)
    print("Modelo cargado exitosamente.")

    # Calentamiento de la GPU para este modelo
    print("Realizando calentamiento de la GPU...")
    _ = model.predict(test_images_batch[:1], verbose=0)
    print("Calentamiento completado.")

    # Medición del tiempo
    print("\n--- ¡Iniciando medición de FPS! ---")
    start_time = time.time()
    _ = model.predict(test_images_batch, verbose=0)
    end_time = time.time()

    # Cálculo de resultados
    total_time = end_time - start_time
    num_images = len(test_images_batch)
    fps = num_images / total_time
    time_per_image_ms = (total_time / num_images) * 1000

    print("\n--- 📊 Resultados ---")
    print(f"Modelo: {model_name}")
    print(f"Imágenes procesadas: {num_images}")
    print(f"Tiempo total: {total_time:.2f} segundos")
    print(f"Tiempo por imagen: {time_per_image_ms:.2f} ms")
    print(f"Rendimiento (FPS): {fps:.2f} FPS")
    print("--------------------")

--- Iniciando Test de Inferencia de FPS en Google Colab ---
✅ GPU detectada: 0

Cargando 100 imágenes de prueba...
Imágenes cargadas. Shape del batch: (100, 128, 128, 3)

Analizando: Modelo S (EfficientNetV2-S)
Cargando modelo: efficient_weed_model_S_best.keras...
Modelo cargado exitosamente.
Realizando calentamiento de la GPU...
Calentamiento completado.

--- ¡Iniciando medición de FPS! ---

--- 📊 Resultados ---
Modelo: Modelo S (EfficientNetV2-S)
Imágenes procesadas: 100
Tiempo total: 19.02 segundos
Tiempo por imagen: 190.22 ms
Rendimiento (FPS): 5.26 FPS
--------------------

Analizando: Modelo B0 (EfficientNetV2-B0)
Cargando modelo: efficient_weed_model_B0_best.keras...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'aspp_module_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'deformable_attention_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Modelo cargado exitosamente.
Realizando calentamiento de la GPU...


Calentamiento completado.

--- ¡Iniciando medición de FPS! ---



--- 📊 Resultados ---
Modelo: Modelo B0 (EfficientNetV2-B0)
Imágenes procesadas: 100
Tiempo total: 26.05 segundos
Tiempo por imagen: 260.47 ms
Rendimiento (FPS): 3.84 FPS
--------------------
